In [1]:
import contextlib
import os

import pandas as pd
from constants import distinct_samples
from SALib.sample.sobol import sample

from mesa.examples.advanced.epstein_civil_violence.agents import (
    CitizenState,
)

# Generate points

In [2]:
param_tuples = [
    ("legitimacy", [0.1, 1.0]),
    ("active_threshold", [0.0, 0.9]),
    ("reversion_rate", [0.0, 1.0]),
    ("prob_quiet", [0.0, 0.5]),
]

param_names, param_bounds = zip(*param_tuples)
param_names = list(param_names)
param_bounds = list(param_bounds)

problem = {
    "num_vars": len(param_names),
    "names": param_names,
    "bounds": param_bounds,
}

model_reporters = {
    "active": CitizenState.ACTIVE.name,
    "quiet": CitizenState.QUIET.name,
    "arrested": CitizenState.ARRESTED.name,
}

params = sample(problem, distinct_samples)
data = pd.DataFrame(params, columns=param_names)

with contextlib.suppress(FileExistsError):
    os.mkdir("inputs_new")

x = 0
pointer = 0
slice_size = 256
while pointer + slice_size < len(data):
    slice = data[pointer : pointer + slice_size]
    slice.to_csv(f"inputs_new/data_{x:04}.csv")
    x += 1
    pointer += slice_size

slice = data[pointer:]
slice.to_csv(f"inputs_new/data_{x:04}.csv")